In [1]:
from fbprophet import Prophet
import pandas as pd
from matplotlib import pyplot as plt
import warnings
import numpy as np
import os
os.environ['NUMEXPR_MAX_THREADS'] = '16'
os.environ['NUMEXPR_NUM_THREADS'] = '10'

In [2]:
sku_models = {}
sku_predictions = {}

def fit_prohpet_model(x):
    df = pd.DataFrame({'ds':x.week, 'y':x.units_sold, 'total_price':x.total_price, 'base_price':x.base_price, 'is_featured_sku':x.is_featured_sku , 'is_display_sku': x.is_display_sku, 'total_price_by_max_total_price':x.total_price_by_max_total_price})
    
    discounts_train = pd.DataFrame({
      'holiday': 'discount',
      'ds': df.loc[df.total_price < df.base_price, 'ds'],
      'lower_window': 0,
      'upper_window': 1,
    })
    
    discounts_test = pd.DataFrame({
      'holiday': 'discount',
      'ds': df_test.loc[(df_test.store_id == x.name[0]) & (df_test.sku_id == x.name[1]) & (df_test.total_price < df_test.base_price), 'week'],
      'lower_window': 0,
      'upper_window': 1,
    })
    
    
    featured_train = pd.DataFrame({
      'holiday': 'featured',
      'ds': df.loc[df.is_featured_sku == 1, 'ds'],
      'lower_window': 0,
      'upper_window': 1,
    })
    
    featured_test = pd.DataFrame({
      'holiday': 'featured',
      'ds': df_test.loc[(df_test.store_id == x.name[0]) & (df_test.sku_id == x.name[1]) & (df_test.is_featured_sku == 1), 'week'],
      'lower_window': 0,
      'upper_window': 1,
    })
    
    displayed_train = pd.DataFrame({
      'holiday': 'displayed',
      'ds': df.loc[df.is_display_sku == 1, 'ds'],
      'lower_window': 0,
      'upper_window': 1,
    })
    
    displayed_test = pd.DataFrame({
      'holiday': 'displayed',
      'ds': df_test.loc[(df_test.store_id == x.name[0]) & (df_test.sku_id == x.name[1]) & (df_test.is_display_sku == 1), 'week'],
      'lower_window': 0,
      'upper_window': 1,
    })
    
    special_events = pd.concat((discounts_train, discounts_test, featured_train, featured_test, displayed_train, displayed_test))
    #     m = Prophet()
    m = Prophet(holidays=special_events)
    #     m.add_seasonality(name='monthly', period=4, fourier_order=4)
    m.add_regressor('total_price')
    m.add_regressor('base_price')
    m.add_regressor('total_price_by_max_total_price')
#     m.add_regressor('is_featured_sku')
#     m.add_regressor('is_display_sku')
    m.fit(df)
    future = df_test.loc[(df_test.store_id == x.name[0]) & (df_test.sku_id == x.name[1]), ['week', 'total_price', 'base_price', 'is_featured_sku', 'is_display_sku', 'total_price_by_max_total_price']].rename(columns={'week':'ds'})
#     future = m.make_future_dataframe(periods=12)
    forecast = m.predict(future)
    sku_models[x.name] = m
    sku_predictions[x.name] = forecast
    return 1

In [3]:
df_train = pd.read_csv('data/train.csv', header=0)
df_test = pd.read_csv('data/test.csv', header=0)
submission = pd.read_csv('data/sample_submission.csv', header=0)

In [4]:
df_train.shape, df_test.shape

((150150, 9), (13860, 8))

In [5]:
df_train.apply(lambda x: pd.isna(x).sum(), axis=0), df_test.apply(lambda x: pd.isna(x).sum(), axis=0)

(record_ID          0
 week               0
 store_id           0
 sku_id             0
 total_price        1
 base_price         0
 is_featured_sku    0
 is_display_sku     0
 units_sold         0
 dtype: int64,
 record_ID          0
 week               0
 store_id           0
 sku_id             0
 total_price        0
 base_price         0
 is_featured_sku    0
 is_display_sku     0
 dtype: int64)

In [6]:
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)

In [7]:
df_store_sku_max_base_price = df_train.groupby(['store_id', 'sku_id']).agg({'base_price': 'max'}).reset_index()
df_store_sku_max_base_price.columns = ['store_id', 'sku_id', 'max_base_price']
print(df_store_sku_max_base_price)

df_store_sku_max_total_price = df_train.groupby(['store_id', 'sku_id']).agg({'total_price': 'max'}).reset_index()
df_store_sku_max_total_price.columns = ['store_id', 'sku_id', 'max_total_price']
print(df_store_sku_max_total_price)

      store_id  sku_id  max_base_price
0         8023  216233        140.3625
1         8023  216418        119.7000
2         8023  216419        119.7000
3         8023  216425        141.0750
4         8023  217390        174.5625
...        ...     ...             ...
1150      9984  300021         89.0625
1151      9984  320485        205.9125
1152      9984  378934        205.9125
1153      9984  547934        191.6625
1154      9984  679023        234.4125

[1155 rows x 3 columns]
      store_id  sku_id  max_total_price
0         8023  216233         140.3625
1         8023  216418         119.7000
2         8023  216419         119.7000
3         8023  216425         141.0750
4         8023  217390         174.5625
...        ...     ...              ...
1150      9984  300021          89.0625
1151      9984  320485         205.9125
1152      9984  378934         205.9125
1153      9984  547934         191.6625
1154      9984  679023         234.4125

[1155 rows x 3 columns]


In [8]:
df_train = pd.merge(df_train, df_store_sku_max_total_price, on=['store_id', 'sku_id'])
df_train['total_price_by_max_total_price'] = df_train['total_price']/df_train['max_total_price']

df_test = pd.merge(df_test, df_store_sku_max_total_price, on=['store_id', 'sku_id'])
df_test['total_price_by_max_total_price'] = df_test['total_price']/df_test['max_total_price']

df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)

In [9]:
df_train.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,max_total_price,total_price_by_max_total_price
0,1,17/01/11,8091,216418,99.0375,111.8625,0,0,20,120.4125,0.822485
1,1641,24/01/11,8091,216418,99.0375,111.8625,0,0,34,120.4125,0.822485
2,3277,31/01/11,8091,216418,96.9000,96.9000,0,0,10,120.4125,0.804734
3,4913,07/02/11,8091,216418,98.3250,98.3250,0,0,17,120.4125,0.816568
4,6553,14/02/11,8091,216418,106.8750,106.8750,0,0,24,120.4125,0.887574


In [10]:
df_train.shape, df_test.shape

((150150, 11), (13860, 10))

In [11]:
df_train.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,max_total_price,total_price_by_max_total_price
0,1,17/01/11,8091,216418,99.0375,111.8625,0,0,20,120.4125,0.822485
1,1641,24/01/11,8091,216418,99.0375,111.8625,0,0,34,120.4125,0.822485
2,3277,31/01/11,8091,216418,96.9000,96.9000,0,0,10,120.4125,0.804734
3,4913,07/02/11,8091,216418,98.3250,98.3250,0,0,17,120.4125,0.816568
4,6553,14/02/11,8091,216418,106.8750,106.8750,0,0,24,120.4125,0.887574


In [12]:
df_train['units_sold'] = np.log(df_train['units_sold'])

In [13]:
df_train['week'].sample(100)

122103    05/09/11
102870    24/10/11
10289     30/05/11
126295    17/04/12
48730     26/02/13
            ...   
126622    31/01/11
122237    03/10/11
97584     28/08/12
131927    05/02/13
26590     22/05/12
Name: week, Length: 100, dtype: object

In [14]:
df_train['week'] = pd.to_datetime(df_train['week'], format='%d/%m/%y')
df_test['week'] = pd.to_datetime(df_test['week'], format='%d/%m/%y')

In [15]:
df_train.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].sum()

150150

In [16]:
df_train.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].min(), \
df_train.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].max()

(130, 130)

In [17]:
df_test.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].min(), \
df_test.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].max()

(12, 12)

In [18]:
print(df_train['sku_id'].unique().shape[0], df_test['sku_id'].unique().shape[0])

28 28


In [19]:
%%time
df_train.groupby(['store_id', 'sku_id']).apply(fit_prohpet_model)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

...

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

CPU times: user 28min 25s, sys: 9.09 s, total: 28min 34s
Wall time: 28min 35s


store_id  sku_id
8023      216233    1
          216418    1
          216419    1
          216425    1
          217390    1
                   ..
9984      300021    1
          320485    1
          378934    1
          547934    1
          679023    1
Length: 1155, dtype: int64

In [20]:
len(sku_models)

1155

In [21]:
list(sku_predictions.items())[0][1]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,base_price,...,total_price_by_max_total_price,total_price_by_max_total_price_lower,total_price_by_max_total_price_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2013-07-16,4.699788,4.352442,4.931583,4.699788,4.699788,-0.056046,-0.056046,-0.056046,0.027244,...,-0.088454,-0.088454,-0.088454,0.093618,0.093618,0.093618,0.0,0.0,0.0,4.643743
1,2013-07-23,4.699687,5.000237,5.580520,4.699679,4.699692,0.588184,0.588184,0.588184,0.027244,...,0.064537,0.064537,0.064537,0.107830,0.107830,0.107830,0.0,0.0,0.0,5.287871
2,2013-07-30,4.699585,4.959254,5.572136,4.699563,4.699602,0.567421,0.567421,0.567421,0.027244,...,0.059602,0.059602,0.059602,0.096938,0.096938,0.096938,0.0,0.0,0.0,5.267007
3,2013-08-06,4.699484,4.755142,5.402023,4.699445,4.699516,0.363705,0.363705,0.363705,0.027244,...,0.059602,0.059602,0.059602,0.111002,0.111002,0.111002,0.0,0.0,0.0,5.063189
4,2013-08-13,4.699383,4.413642,5.001462,4.699327,4.699436,0.009040,0.009040,0.009040,0.027244,...,-0.088454,-0.088454,-0.088454,0.158704,0.158704,0.158704,0.0,0.0,0.0,4.708423
5,2013-08-20,4.699281,4.467948,5.088140,4.699206,4.699360,0.079055,0.079055,0.079055,0.014337,...,-0.063778,-0.063778,-0.063778,0.192274,0.192274,0.192274,0.0,0.0,0.0,4.778336
6,2013-08-27,4.699180,4.410688,5.043163,4.699081,4.699283,0.047485,0.047485,0.047485,0.019500,...,-0.073648,-0.073648,-0.073648,0.175281,0.175281,0.175281,0.0,0.0,0.0,4.746664
7,2013-09-03,4.699078,4.430500,5.001421,4.698953,4.699214,0.013568,0.013568,0.013568,0.016918,...,-0.068713,-0.068713,-0.068713,0.134076,0.134076,0.134076,0.0,0.0,0.0,4.712646
8,2013-09-10,4.698977,4.359159,4.982724,4.698815,4.699135,-0.020660,-0.020660,-0.020660,0.022081,...,-0.078583,-0.078583,-0.078583,0.114425,0.114425,0.114425,0.0,0.0,0.0,4.678316
9,2013-09-17,4.698876,4.335948,4.941790,4.698681,4.699062,-0.054628,-0.054628,-0.054628,0.029825,...,-0.093389,-0.093389,-0.093389,0.102325,0.102325,0.102325,0.0,0.0,0.0,4.644248


In [22]:
preds = None
for store, sku in sku_predictions.keys():
    preds_ = pd.DataFrame({'store_id': store, 'sku_id': sku, 'week': sku_predictions[(store, sku)]['ds'], 'units_sold': sku_predictions[(store, sku)]['yhat']})
    if preds is None:
        preds = preds_
    else:
        preds = pd.concat([preds, preds_])

In [23]:
preds.shape

(13860, 4)

In [24]:
df_train.shape[0] + df_test.shape[0]

164010

In [25]:
submission = pd.merge(df_test[['record_ID', 'week', 'store_id', 'sku_id']], preds, on=['week', 'store_id', 'sku_id'])

In [26]:
submission.shape

(13860, 5)

In [27]:
df_test.shape

(13860, 10)

In [28]:
submission['units_sold'] = np.exp(submission['units_sold'])

In [29]:
submission[['record_ID', 'units_sold']].to_csv('fb_prophet_baseline.csv', index=None)